In [54]:
#Import required libraries
import pandas as pd
# For displaying search results in a table
from IPython.display import HTML, display
from os.path import join as pjoin
import csv

# For persisting indexes in an external file
import pickle
import math
import heapq
from datetime import datetime
from pathlib import Path


import nltk
import csv
import re
import os

# For word tokenization
from nltk.tokenize import RegexpTokenizer
# For stop words list
from nltk.corpus import stopwords
# For word stemming
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+|\$')
ps = SnowballStemmer('english')

# Path to the current working directory to refer to all the files relatively
my_path = os.path.dirname(os.path.realpath('__file__'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vamsigunturi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 1: Data

In [55]:
#Reading the main CSV file
m = pd.read_csv("Airbnb_Texas_Rentals.csv",encoding = 'utf8')
doc_len = len(m)

# These are commented because we already processed the reviews in to invidual CSV file per review
#we found words like '\\n' in the dataset so we cleaned it
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\t',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\r',  ' ', regex=True)

#create tsv files,we droped the first column that was a prroblem
#m = m.drop(['Unnamed: 0'], axis=1)

# Step 2: Create documents

In [56]:
# These lines are commented because we already have the individual csv file per review,
# after first time we executed this section of code. so we no longer need to execute this every time
# Create a separate csv file for every row in the reviews csv file
#for i in range(len(m)):
#    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(i) + '.tsv', 'w', newline='',encoding='utf-8') as output:
#        tsv_output = csv.writer(output, delimiter='\t')
#        tsv_output.writerow(m.iloc[i])

# Step 3: Search Engine

## 3.1) Conjunctive query

### 3.1.1) Create your index!

In [57]:
review_content_persist = {}
vocabulary_persist = {}
words_persist = {}

# Retrieving persisted information for review content and word map
# Please create a directory(in your current working directory) with name 'indexes'  
content_file = Path(os.path.join(my_path, "indexes/review_content.pkl"))
vocabulary_file = Path(os.path.join(my_path, "indexes/vocabulary.pkl"))
words_file = Path(os.path.join(my_path, "indexes/words.pkl"))


# Retrieving already persisted information

# Check if the index file exists, 
#if yes load the previously persisted indexes and content
if content_file.is_file():
    with open(content_file, "rb") as review_content:
        review_content_persist = pickle.load(review_content)
        review_content.close()
        
# Check if the vocabulary file exists, 
#if yes load the previously persisted vocabulary
if vocabulary_file.is_file():
    with open(vocabulary_file, "rb") as vocabulary:
        vocabulary_persist = pickle.load(vocabulary)
        vocabulary.close()
        
# Check if the words file exists, 
#if yes load the previously persisted words
if words_file.is_file():
    with open(words_file, "rb") as words:
        words_persist = pickle.load(words)
        words.close()

if(len(review_content_persist.keys()) == 0):
    
    review_word_map = {}
    
    # We reach here if we don't have indexes already present
    print("Indexes are being created")
    
    #we create the vocabulary of preprocessed documents,but we don't modify the documents because we''l use them in search engine
    
    for i in range(doc_len):
        with open(os.path.join(my_path, 'docu_hw3/doc_' + str(i) + '.tsv'),encoding='utf8') as tsvfile:
             tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
        
        # For review title
        l1 = tsvreader[0][4]
        
        # For review content
        l2 = tsvreader[0][7]
        
        l = l1+ ' ' +l2
        l = l.lower()
        l = tokenizer.tokenize(l)
        
        # This array will contain all the valid words in a given review after removing 
        # all the stop words, punctuations, stemming etc..,, we will use this information
        # to find out the term frequency there by tf-idf values
        file_words = []
        
        for r in l :
            if not r in stop_words:
                sr = ps.stem(r)
                
                file_words.append(sr)
                
                if not  sr in review_word_map:
                    review_word_map[sr] = [i]
                else:
                    review_word_map[sr]+=[i]
                    
                    
        review_content_persist[i] = ' '.join(file_words)
    
    # Saving the content and indexes for the first time
    # We made use of pickel python module
    #Saving content dictionary
    with open(content_file, "wb") as review_content:
        pickle.dump(review_content_persist, review_content)
        review_content.close()
    
    # Word and Vocabulary indexes based on word map
    c = 0
    for key in review_word_map:
        words_persist[key] = c
        vocabulary_persist[c] = review_word_map[key]
        c += 1
    
    #Save vocabulary and words
    with open(vocabulary_file, "wb") as vocabulary:
        pickle.dump(vocabulary_persist, vocabulary)
        vocabulary.close()
        
   
    with open(words_file, "wb") as words:
        pickle.dump(words_persist, words)
        words.close()
    
    
                

            

### 3.1.2) Execute the query

In [58]:
word = input('Enter a search query: ')

# Cleaning user input similar to what we did for creating indexes for words
def clean_input(w):
    w_list = []
    w = w.lower()
    w = tokenizer.tokenize(w)
    # Check if we need to do any other preprocessing to improve the efficiency of search results
    
    for r in w :
        if not r in stop_words:
            sr = ps.stem(r)
            if not  sr in w_list:
                w_list.append(sr)
    return w_list

# Show search results in tabular format
def show_results(results, doc_list, isScore):
    
    if(len(doc_list)):
        print('Found ' + str(len(doc_list))  + ' matching reviews to your query')
    
    if(len(results)):
        if(isScore):
            tableFormat = '<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th><th>Score</th></tr><tr>{}</tr></table>'
        else:
            tableFormat = '<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr><tr>{}</tr></table>'
        
        
        display(HTML(tableFormat.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results)
)))
    else:
        display(HTML('<h1>No results found. Please try a different query</h1>'))
    


word_list = clean_input(word)

print("Cleaned word: ", word_list)

list_doc_list = []

for w in word_list:
        doc_list = []
        
        if w in words_persist: 
            doc_list = vocabulary_persist[words_persist[w]]
            
        list_doc_list.append(doc_list)  

# Initially assinging the list intersection to the matching documents of first word
list_intersect = list_doc_list[0]

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
        
for docList in list_doc_list:
    list_intersect = intersection(list_intersect, docList)
    
results = []

# Removing the duplicates in the document intersection
list_intersect = list(set(list_intersect))

i_len = len(list_intersect)

if(i_len):
    
    r_limit = 10
    
    if(i_len < 10):
        r_limit = i_len
    
    # Showing at most ten results
    for doc in list_intersect[:r_limit]:
        
        # Reading each document based on document id in list intersect 
        with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
             tsvreader = list(csv.reader(tsvfile, delimiter="\t"))

        title = tsvreader[0][7]        
        description = tsvreader[0][4]
        city = tsvreader[0][2]
        url = tsvreader[0][8]

        results.append([title, description, city, url])

# Displaying the results
show_results(results, list_intersect, False)

Enter a search query: beautiful house
Cleaned word:  ['beauti', 'hous']
Found 797 matching reviews to your query


Title,Description,City,URL
Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. Amazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. Featuring an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more European inspired kitchen and “top of the line” decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX
Friendly Private Room in َQuiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX
East Austin Hillside Gem,"Beautiful and modern 3Br, 2.5Ba located minutes from downtown. Amenities include front deck, back patio, peaceful backyard with tiered gardens and fire pits. There is also gym equipment, wifi, Xbox, and cable with HBO, Showtime, Starz, etc. There is free offstreet parking for two cars. Perfect location for families(with kids or furry friends,) couples, business travelers or friends looking for a spacious and comfortable location with plenty of amenities. This house has beautiful sunset views of downtown Austin just 6 miles from Darrell K Royal Stadium, 8 miles from Zilker Park and 14 miles from Circuit of the Americas.",Austin,https://www.airbnb.com/rooms/17555039?location=Bastrop%20County%2C%20TX
A Cozy Home with a Beautiful Nature Views,"This home is on the North Side of San Antonio, 3 minutes away from a great golfing course and a JW Marriott Resort and Spa. This is a great stopping place for a family and comes with all amenities from cable, internet to a laundry room and 2 bathrooms. This house is against a green belt so you can enjoy the back yard and nature.This area is safe and my neighbors are also great! Hope you enjoy your stay and more so enjoy San Antonio!",San Antonio,https://www.airbnb.com/rooms/19190311?location=Cibolo%2C%20TX
Cottage in the Oaks,"This cottage sits on a 2 acre lot with beautiful swayed oaks. Large front patio which is GREAT for entertaining or just relaxing and watching the sun set. This house has a backyard with a small deck and ivy covered oaks. Inside, there are two bedrooms and two bathrooms. Smart TV in living room with ROKU technology. Additional smart TV in master bedroom. Fully stocked kitchen (just bring your own groceries!) Washer and dryer available. Only 200 yards from the bay! Great for families.",Rockport,https://www.airbnb.com/rooms/16585732?location=Bayside%2C%20TX
"The Woodlands, BEAUTIFUL HOME, 1 Floor, 2 BT, 3 BD","Attractions: The Woodlands, incredible views, golf courses, theme parks with tennis courts and lakes, pools, cycle paths, tranquility and quality of life for the family. You will love my house because of its cozy spaces with high ceilings in an excellent location, beautiful views, and proximity to the great shopping areas. It is a single story house with a large garden; ideal for couples, business travelers, and families with children.",Spring,https://www.airbnb.com/rooms/13065223?location=Conroe%2C%20TX
Super Bowl Bungalow Near Downtown & Stadium!!!,"Super Bowl Ready!!! Close to Downtown, Midtown, Montrose and the Stadium! Charming corner lot bungalow in quiet historic neighborhood. Jogging trail along bayou. Across from beautiful cemetery. One private bedroom with a queen and lock on door. Back bedroom with pull out queen and back door entrance. Middle bedroom with queen (must pass through middle bedroom to access back bedroom). Two large couches in living. Small house with only one bathroom that can sleep 8 adults.",Houston,https://www.airbnb.com/rooms/17

## 3.2) Conjunctive query & Ranking score

### 3.2.1) Inverted index

In [59]:
iindex_tf_idf_persist = {}


# Check if the index file exists, if yes load the previously persisted indexes and content
# Please create a directory(in your current working directory) with name indexes  
index_file = Path(os.path.join(my_path, "indexes/iindex_tf_idf.pkl"))

# Check if the index file exists, if yes load the previously persisted indexes
if index_file.is_file():
    # Retriving precreated inverted indexes
    with open(index_file, "rb") as iindex_tf_idf:
        iindex_tf_idf_persist = pickle.load(iindex_tf_idf)
        iindex_tf_idf.close()
        
    
if(len(iindex_tf_idf_persist.keys()) == 0):
    
    print("Inverted Indexes are being calculated")

    word_iindex = {}

    #Creating inverted index using tf-idf and consine similarity
    for word in words_persist:
        word_doc_list = vocabulary_persist[words_persist[word]]
        word_iindex[word] = []

        # Store indexes based on number of times a particular word is present in a given document
        for doc in word_doc_list:
            doc_content = review_content_persist[doc]
            # Pushing the term frequency with document id
            word_iindex[word].append([doc, doc_content.split().count(word)])

    # Store indexes based on tf-idf
    docs_length = len(review_content_persist.keys())
    iindex_tf_idf_persist = word_iindex

    for key, word in iindex_tf_idf_persist.items():
        # find out the relative importance of a particular terms relating it to document count
        idf= math.log10( docs_length / len(word) )

        for elem in word:
            # Add the document score corresponding to a particular term which we then use in the 
            # search results ranking of documents
            elem[1] = idf * elem[1]
    
    # Persisting the indexes calculated 
    with open(index_file, "wb") as iindex_tf_idf:
        pickle.dump(iindex_tf_idf_persist, iindex_tf_idf)
        iindex_tf_idf.close()
        


### 3.2.2) Execute the query

In [60]:
dict_qcos = {}
dict_norm = {}

#print("Search started")

# Need to remove this 100 hardcoding
for doc in list_intersect[:100]:
    
    num = 0
    
    #print("Current document ID: " + str(doc))
    
    # Calculating numerator of the cosine similarity equation
    
    for word in word_list:
        w_index = iindex_tf_idf_persist[word]
        w_i_len = len(w_index)
        for i in range(w_i_len):
            if w_index[i][0] == doc: 
                num +=  w_index[i][1]
                
    dict_qcos[doc]=num

    
    # Calculating denominator of the cosine similarity equation
    norm = 0
    for word in iindex_tf_idf_persist.values():
        for i in range(len(word)):
            if word[i][0] == doc:
                norm +=  word[i][1]**2
    
    dict_norm[doc]=math.sqrt(norm)

#print("Numerator and Denominator calculated")

# Once numerator and denominator is calculated find the score of each document in the intersection list 
# By applying the consine similarity formala
for doc,num in dict_qcos.items():
    # Eleminating divided by zero problem to check if the normalization value for a document is non-zero
    if dict_norm[doc] != 0:
        dict_qcos[doc] = num/(math.sqrt(len(word_list))*dict_norm[doc])

#print("Cosine similarity done")

In [61]:
# Applying heap data structure to print to top-k documents

h = []
results = []

for doc in dict_qcos.keys():
    
    # Reading the document meta data to print in the search results
    
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    # Pushing the document information to heap data structure
    
    heapq.heappush(h,(dict_qcos[doc], title, description, city, url))

#print("Applying Heap")    
# Applying max heap algorithm
heapq._heapify_max(h)

limit = 10
doc_len = len(dict_qcos.keys())

if(doc_len < 10):
    limit = doc_len

for i in range(limit):
    # Popping the document with maximum score at every step and adding it to the result list
    
    # Since the data structure used in heap is a tuple, we convert it in to list for ease of manipulation
    results.append(list(heapq.heappop(h)))
    
    # re-applying the max heap algorithm
    heapq._heapify_max(h)

#print("Applying Heap")  

#print(results)
results_formatted = []

for i in results:
    
    first = round(i.pop(0), 4)
    i.append(first) 
    #first = round(float(results[i].pop(0)), 4)
    #results[i] = results[i].append(first)
    results_formatted.append(i)


# Displaying the results
show_results(results_formatted, [], True)

Title,Description,City,URL,Score
Quite Spacious Private Room in Beautiful House,"Our house is in a small and quite family neighborhood. We have a very spacious family home. My wife and I work a lot and are rarely here but keep the house very clean. The house is located right off I-10 while quite a bit of restaurants, shopping centers, and enertainment close to our house.",Houston,https://www.airbnb.com/rooms/18784719?location=Cinco%20Ranch%2C%20TX,0.6543
My house is big and quite.,My house is a Mexican rustic decorate. Big room and clean. Clean and beautiful house in a quiet neighborhood nearby public transportation.,San Antonio,https://www.airbnb.com/rooms/7378323?location=Castroville%2C%20TX,0.4993
River Run,"River Run is a gorgeous vacation rental house located in Concan, Texas. It is in the beautiful 18 hole golf course in Concan. It has over 3105 sq. ft. + of space for you and your guests and is a single level house. It is a 5 bedroom 5 bathroom house that sleeps 28 people. There is a private bedroom and bathroom located by the cabana area in back of the house.",Concan,https://www.airbnb.com/rooms/17808131?location=Concan%2C%20TX,0.3997
The Serenity Rose House: Large Home up to 20 ppl,"Stay in a large county home in Conroe, TX! Up to 20 guest can be accommodated in this house. The property edge is on the beautiful wooded Egypt Creek which meanders close by! The house is ten minutes away from lake Conroe and 20 minutes from the Woodlands mall. Spacious home with large rooms and beautiful views of nearby trees and occasional deer along the creek.",Conroe,https://www.airbnb.com/rooms/18666670?location=Conroe%2C%20TX,0.3325
Casa de Creosote. Off-grid Cabin + RV and Camping,This is a romantic studio style cabin/house with a lofted bedroom in the second floor peek of the a-frame. This house offers privacy and solitude being that it is on a 10 acre piece of land on Terlingua Ranch with beautiful 360 degree views of the Christmas Mountains and Nine Point Mesa. I built this house as a creative space to make music and enjoy the Big Bend Region. The beautiful wooden porch with multiple hammocks and outdoor couch is great for watching sunsets. Molly is my co-host.,Terlingua,https://www.airbnb.com/rooms/16895505?location=Big%20Bend%20National%20Park%2C%20TX,0.3087
"The Woodlands, BEAUTIFUL HOME, 1 Floor, 2 BT, 3 BD","Attractions: The Woodlands, incredible views, golf courses, theme parks with tennis courts and lakes, pools, cycle paths, tranquility and quality of life for the family. You will love my house because of its cozy spaces with high ceilings in an excellent location, beautiful views, and proximity to the great shopping areas. It is a single story house with a large garden; ideal for couples, business travelers, and families with children.",Spring,https://www.airbnb.com/rooms/13065223?location=Conroe%2C%20TX,0.2813
Beautiful Solar Powered Lake House,"Completely renovated luxury lake house with modern design, solar panels, and lots of upgrades at University Hills located on a quiet street!! 15 mins to downtown!! Beautiful new kitchen!! Open floor plan great for entertainment and gorgeous patio with BBQ Grill!! 4 car parking space!!",Austin,https://www.airbnb.com/rooms/11351108?location=Coupland%2C%20TX,0.281
Enjoy our BEAUTIFUL new home in The Heights!!,"Our beautiful home will allow you to enjoy everything about our city. You'll only be a few miles from downtown and walking distance to excellent food and wine bars or relax on our patio. This house is very clean, comfortable and has everything you'll need to enjoy your stay with plenty of room for everyone in your party.",Houston,https://www.airbnb.com/rooms/16932061?location=Atascocita%2C%20TX,0.2758
Lake LBJ Lake House,"Large Beautiful House on Lake LBJ with 175 ft of water frontage and a large boat house with a partially covered 500 sq. foot party deck with a 60 inch LED TV. SUPs and kayaks provided free. Boat, jet skis for rent. exactly 1 hour from DT Austin",Horseshoe Bay

# Step 4: Define a new score!

In [62]:
av_r_n_u = 0
bedr_c_u = 1000
d_list_u = datetime.strptime('January 1900', '%B %Y')
city_u = 'NA'
one_m_dif = datetime.strptime('February 2015', '%B %Y')- datetime.strptime('January 2015', '%B %Y')
two_m_dif = datetime.strptime('March 2015', '%B %Y')- datetime.strptime('January 2015', '%B %Y')
try:
    av_r_n_u = int(input('please, enter maximum price that you can pay(e.g. 10): '))
except:
    print('No price suggested')
    pass
try:
    bedr_c_u = int(input('please, enter the number of bedrooms that you need(e.g 5): '))
except:
    print('No indication about the needed number of bedrooms')
    pass
try:
    d_list_u = datetime.strptime(input('please, enter your needed period in this form,Month year(e.g. February 2015): '), '%B %Y')
except:
    print('No period selected')
    pass
try:
    city_u = input('please enter the city you want to reside(e.g. Austin): ')
except:
    print('No city was indicated')
    pass
list_ord =[]

n_h = []

for doc in list_intersect:
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    price = list(tsvreader[0][0])
    eff_price = ''
    for i in range(1,len(price)):
        eff_price += price[i]
    
    
    try:
        av_r_n = int(eff_price)
        bedr_c = int(tsvreader[0][1])
    except ValueError:
        av_r_n = 1000000
        bedr_c = 1
    
    d_list = datetime.strptime(tsvreader[0][3],'%B %Y')
    sum_values = 0
    if av_r_n >= 2*av_r_n_u :
               sum_values += 0          
    elif av_r_n < 2*av_r_n_u and av_r_n >= 1.75*av_r_n_u:
               sum_values += 0.25
    elif  av_r_n < 1.75*av_r_n_u and av_r_n >= 1.5*av_r_n_u:
               sum_values += 0.50  
    elif av_r_n < 1.5*av_r_n_u and av_r_n >= 1.25*av_r_n_u:
               sum_values += 0.75 
    elif av_r_n < 1.25*av_r_n_u and av_r_n >= av_r_n_u:
               sum_values += 1           
    elif av_r_n < av_r_n_u and av_r_n >=0.75*av_r_n_u:
               sum_values += 1.25
    elif av_r_n < 0.75*av_r_n_u and av_r_n >=0.5*av_r_n_u :
               sum_values += 1.5
    elif av_r_n < 0.5*av_r_n_u :
               sum_values += 2.
    
    
    if bedr_c >= bedr_c_u:
               sum_values += 1
    elif bedr_c == bedr_c_u-1:
               sum_values += 0.75
    elif bedr_c == bedr_c_u-2:
               sum_values += 0.5
    elif bedr_c <= bedr_c_u-3:
               sum_values += 0 #only to make index easier to understand , no practical effect
    
    if d_list == d_list_u:
               sum_values += 1
    elif d_list - d_list_u == one_m_dif or d_list - d_list_u == -one_m_dif :
               sum_values += 0.5
    elif d_list - d_list_u == two_m_dif or d_list - d_list_u == -two_m_dif :
               sum_values += 0.25
    else:
        sum_values += 0 #only to make index easier to understand , no practical effect
    
    if city_u == city :
        sum_values += 0.5
    
    score = round(sum_values/4.5,2)
    
    #list_ord.append([title, description, city, url, str(score),score])
    
    # Apply heap to the search results based on score value
    heapq.heappush(n_h,(score, title, description, city, url))

results = []

# Applying heapify max algorithm
heapq._heapify_max(n_h)

limit = 10
d_len = len(list_intersect)

if(d_len < 10):
    limit = d_len

for i in range(limit):
    results.append(list(heapq.heappop(n_h)))
    heapq._heapify_max(n_h)

#print("Applying Heap")  

#print(results)
results_formatted = []

for i in results:
    first = round(i.pop(0), 4)
    i.append(first) 
    results_formatted.append(i)
    

# Displaying the results
show_results(results_formatted, [], True)

please, enter maximum price that you can pay(e.g. 10): 30
please, enter the number of bedrooms that you need(e.g 5): 5
please, enter your needed period in this form,Month year(e.g. February 2015): February 2015
please enter the city you want to reside(e.g. Austin): Austin


Title,Description,City,URL,Score
Jolly Cabin,"Property is in quiet, wooded neighborhood and backs up to Grapevine Lake. House has private trail to the famous Northshore Mountain Bike and Running Trails. Close to the new Lakeside DFW community. This is a perfect getaway for Mountain Bikers, Runners and Hikers. Beautiful, rustic home that is clean and updated. Explore the trails in your backyard, relax around the pool, or walk to nearby Lakeside DFW for dinning and movie.",Flower Mound,https://www.airbnb.com/rooms/14086666?location=Argyle%2C%20TX,0.39
Cozy bedroom in the heart of south Austin!,"Cute single bedroom available in house less than 10 minutes to downtown and 5 from green belt! Has a large backyard to relax quiet mornings by a dry bed. Located in a quiet cul de sac. Full beautiful kitchen and bathroom two steps from room. Fully loaded with towels, bed sheets, toiletries, and some house warming surprises! Dogs do live on property, very well behaved.",Austin,https://www.airbnb.com/rooms/19037156?location=Austin%2C%20TX,0.39
Beautiful house 12 minutes from downtown,"Our house is close to downtown as well as Alamo Drafthouse, Ramen Tatsu-ya, Din Ho Chinese BBQ, Bartlett's, Hanabi Sushi, Top Golf, The Domain restaurants & shopping, Hop Doddy and Pinballz old school arcade. You’ll love our place because we're friendly, the back yard is awesome, the neighborhood is quiet, and you're a 10 minute drive to downtown. There's also a bus station walking distance as well.",Austin,https://www.airbnb.com/rooms/14282701?location=Austin%2C%20TX,0.39
The Center of Divine Light,"renovated Mineral Bath House , Wonderful healing Waters for soaks , Labyrinth in Back yard for Walking meditations. Singing Chimes , FIre Pit , Grill for outdoor cooking, Beautiful clear night sky for star gazing. Hanging sky chairs for rest and relaxation. Massage Therapist on hand if you want to schedule some R and R time for yourself. Mysterious magical Orbs that have visit on occasion. Healing Vibro Accoustic Lounge ( sound vibrating table), quite and peaceful setting meditation areas.",Jacksboro,https://www.airbnb.com/rooms/15479650?location=Bridgeport%2C%20TX,0.33
TEXAS STYLE HACIENDA ON 5 ACRES,"9 Bedrooms, 4 Baths, Sleeps 24, 4 living & 2 dinning areas. Wrap around porch. A beautiful blend of Texas Heritage, California Mission and Santa Fe, NM design and construction creates this unique Ranch-style home with sweeping views of the Texas Hill Country. This impressive house can sleep your entire extended family comfortably. Mexican talavera tile adds color and style throughout the home. This is a unique one of a kind home with a spa-like charm for an relaxing or active holiday.",Austin,https://www.airbnb.com/rooms/18370528?location=Bee%20Cave%2C%20TX,0.33
Sth. Austin SXSW & F1-Perfect Spot!,"We have a great home just South of the Lake, a stone’s throw away from downtown Austin. Importantly, we are a close and direct shot downtown. And, as a bonus, we are surrounded by the beauty of the famous Texas Hill Country! You will indulge in a 3,600 square foot house with a great kitchen (gas stove), two downstairs living areas, and a huge upstairs gameroom with a pool table, plasma TV, and game table. There are 4 bedrooms upstairs with 2 full bathrooms. The bedrooms upstairs have two full beds and two queen beds. The large master bedroom is situated downstairs, and has a nice private sitting area and a king size bed. The master bathroom has a rain shower and a Jacuzzi tub. There is a powder room downstairs in the main hallway. After a day of exploring the area, you can celebrate with a Texas BBQ in the spacious, furnished backyard; and relax. If you don’t feel like cooking, no worries – there are great restaurants nearby. Or, you could simultaneously have dinner and catch a movie at the one-of-a-kind South Austin Alamo Draft House - newly opened in our neighborhood. We are located off a major highway system that links directly to the Airport, Downtown SXSW Hotspots, F

# Bonus Step: Make a nice visualization!

In [63]:
df = pd.read_csv("Airbnb_Texas_Rentals.csv")
df = df.dropna(axis = 0)

In [64]:
lat = float(input('Enter a latitude: '))
lon = float(input('Enter a longitude: '))
dis = float(input('Enter distance range (in Kms): ')) 

Enter a latitude: 30.05
Enter a longitude: -95.03
Enter distance range (in Kms): 30


In [65]:
#!pip install geopy
import folium
from geopy import distance

m = folium.Map(location = [lat, lon], zoom_start = 12)

In [66]:
search = (lat, lon)

for row in df.itertuples():
    if distance.distance(search, (row.latitude, row.longitude)).km <= dis:
        folium.Marker(location = [row.latitude, row.longitude], popup = folium.Popup('<a href=' + row.url + '>' + row.average_rate_per_night + ' </a>')).add_to(m)
        folium.Circle(location = [lat, lon], radius = dis * 1000).add_to(m)
        
m.save('map.html')

In [67]:
m